## Consultando os dados do banco

Uma vez com o banco de dados relacional criado e instanciado no MySQL, podemos utilizá-lo para as mais diversas consultas.

In [14]:
import mysql.connector
import pandas as pd 

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Pedro051220011-",
    database="netflix"
)

cursor = conn.cursor()

In [15]:
def run(query: str):
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    
    return pd.DataFrame(rows, columns=columns)

### Selecionando os 5 primeiros filmes registrados no banco

In [16]:
query = "SELECT title FROM content LIMIT 5";
run(query)

,title
0,Dick Johnson Is Dead
1,Blood & Water
2,Ganglands
3,Jailbirds New Orleans
4,Kota Factory


### Selecionando um filme do nosso interesse e quando ele foi adicionado à Netflix

In [17]:
# pode alterar abaixo para um filme qualquer (em inglês)
movie = "The Conjuring" # invocação do mal
query = 'SELECT title, date_added FROM content WHERE title like "{}%"'.format(movie)

print("Filme está ou não na Netflix? Se sim, temos o título e a data em que foi adicionado ao catálogo.")
run(query)

Filme está ou não na Netflix? Se sim, temos o título e a data em que foi adicionado ao catálogo.


,title,date_added
0,The Conjuring,2021-02-21
1,The Conjuring 2,2021-02-21


### Selecionando os filmes que um diretor de nosso interesse dirigiu

In [18]:
# pode alterar abaixo para um diretor qualquer
director = "Tarantino"
query = 'SELECT title FROM content WHERE id in (SELECT content_id FROM director INNER JOIN content_director ON id=director_id WHERE director_name like "%{}%") AND id in (SELECT content_id FROM content_type INNER JOIN type ON id=type_id WHERE type_name like "Movie")'.format(director)

print(f"Todos os filmes no catálogo da Netflix dirigidos por {director}:")
run(query)

Todos os filmes no catálogo da Netflix dirigidos por Tarantino:


,title
0,Django Unchained
1,The Hateful Eight
2,Inglourious Basterds
3,Jackie Brown
4,Kill Bill: Vol. 1
5,Kill Bill: Vol. 2
6,Pulp Fiction


### Selecionando os TV shows com classificação para adultos que tenham mais do que 8 temporadas

In [19]:
query = 'SELECT title FROM content INNER JOIN tv_show_duration ON id=content_id WHERE id in (SELECT content_id FROM content_type INNER JOIN type ON id=type_id WHERE type_name like "Tv Show") AND id in (SELECT content_id FROM content_rating INNER JOIN rating ON id=rating_id AND name like "TV-MA") AND seasons > 8;'
run(query)

,title
0,The Walking Dead
1,Stargate SG-1
2,American Horror Story
3,Call the Midwife
4,Shameless (U.S.)
5,COMEDIANS of the world
6,Trailer Park Boys
7,Forensic Files
8,Club Friday The Series 6


### Selecionando a quantidade de filmes e de TV shows no catálogo

In [20]:
query = 'SELECT type_name AS "Tipo de Mídia", COUNT(type_id) AS "Quantidade" FROM content_type INNER JOIN type ON id=type_id GROUP BY(type_id)'
run(query)

,Tipo de Mídia,Quantidade
0,Movie,6131
1,TV Show,2676


### Selecionando os atores que atuaram em mais do que 25 filmes

In [21]:
query = 'SELECT actor_name AS "Ator ou atriz", count(id) AS "Atuações" FROM actor INNER JOIN content_cast ON id=actor_id GROUP BY actor_name HAVING count(id) > 25'
run(query)

,Ator ou atriz,Atuações
0,Akshay Kumar,30
1,Rupa Bhimani,31
2,Amitabh Bachchan,28
3,Rajesh Kava,26
4,Paresh Rawal,28
5,Vincent Tong,26
6,Yuki Kaji,29
7,Naseeruddin Shah,32
8,Anupam Kher,43
9,Julie Tejwani,33


### Selecionando a quantidade média de atores nos filmes

In [22]:
query = 'SELECT AVG(actor_count) "Média de atores nos filmes" FROM (SELECT COUNT(actor_id) AS actor_count FROM content_cast WHERE content_id IN (SELECT id FROM content WHERE id IN (SELECT content_id FROM content_type WHERE type_id = 1)) GROUP BY content_id) AS actor_counts'
run(query)

,Média de atores nos filmes
0,7.8630


(i) Pelo menos 3 das consultas devem contemplar opera ̧c ̃ao( ̃oes) de jun ̧c ̃ao( ̃oes), devendo ter pelo
menos uma jun ̧c ̃ao externa (em alguma consulta) e deve-se utilizar mais de uma jun ̧c ̃ao em pelo
menos duas das consultas;

(ii) Pelo menos 2 das consultas devem fazer uso de uma ou mais fun ̧c ̃oes de agrega ̧c ̃ao, sendo que pelo
menos uma delas tamb ́em deve realizar agrupamento; FOI

(iii) Pelo menos 2 das consultas devem fazer uso de subconsultas aninhadas. FOI

### Selecionando os gêneros de algum filme ou Tv Show

In [23]:
content = "Naruto"
query = """SELECT genre_name AS 'Gêneros de {}'
            FROM content
            RIGHT JOIN content_genre on content.id=content_id
            RIGHT JOIN genre ON genre.id=genre_id
            WHERE title like '{}'""".format(content, content)
run(query)

,Gêneros de Naruto
0,International TV Shows
1,Anime Series


### Selecionando filmes ou TV shows com um gênero do nosso interesse

In [24]:
genre = "Horror"
query = "SELECT title FROM content INNER JOIN content_genre ON id=content_id WHERE genre_id in (SELECT id FROM genre WHERE genre_name like '%{}%')".format(genre)
run(query)

,title
0,Midnight Mass
1,Brand New Cherry Flavor
2,RESIDENT EVIL: Infinite Darkness
3,Scream
4,Black Summer
...,...
427,What Lies Beneath
428,Would You Rather
429,XX
430,Yoga Hosers
